# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProject'

experiment=Experiment(ws, experiment_name)

In [3]:
#create compute cluster
amlcompute_cluster_name="cpu-compute"

#verify that cluster does not already exist
try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_DS2_V2",max_nodes=5)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
dataset_path="https://raw.githubusercontent.com/prajwalnagaraja07/Udacity-Capstone-Project/main/heart_failure_clinical_records_dataset.csv"
dataset=TabularDatasetFactory.from_delimited_files(path=dataset_path)

#preview data
df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [5]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'heart_failure_clinical_records_dataset'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("The dataset was found!")

else:
    url = dataset_path
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(ws,key)

data = dataset.to_pandas_dataframe()
train, test = train_test_split(data, shuffle=True)
train.to_csv('train.csv', index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore, 'train.csv')])

The dataset was found!
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.train.automl import AutoMLConfig
import logging
import time


automl_settings = {
    "n_cross_validations":4,
    "primary_metric":'accuracy',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":45,
    "max_concurrent_iterations":5,
   }


automl_config = AutoMLConfig(
    task='classification',
    compute_target='cpu-compute',
    training_data=train,
    label_column_name='DEATH_EVENT',
    featurization = 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
    )

In [7]:
from azureml.core.experiment import Experiment

experiment_name = 'automl_capstone_run'
experiment_=Experiment(ws, experiment_name)


remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpu-compute with default configuration
Running on remote compute: cpu-compute


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject,AutoML_032758d2-dde3-41ac-a05f-7a797dadf0d6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject,AutoML_032758d2-dde3-41ac-a05f-7a797dadf0d6,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_032758d2-dde3-41ac-a05f-7a797dadf0d6',
 'target': 'cpu-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-04-23T10:32:03.922593Z',
 'endTimeUtc': '2023-04-23T10:56:03.721515Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'cpu-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneProject","subscription_id":"779ddc6a-162c-43db-9f05-edaaa1cde3ba","resource_group":"rg-preview-features","workspace_name":"feature-test","region":"germanywestcentral","compute_target":"cpu-compute","spark_ser

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Retrieve and save your best automl model.
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)


print(best_remote_run)
best_remote_run

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection', 'BestRunExplainModel', 'ModelExplanationDataSetSetup', 'PickSurrogateModel', 'EngineeredFeatureExplanations', 'EngineeredFeatureExplanations', 'RawFeaturesExplanations', 'RawFeaturesExplanations', 'BestRunExplainModel']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.', 'Best run model explanations started', 'Model explanations data setup completed', 'Choosing LightGBM as the surrogate model for explanations', 'Computation of engineered features started', 'Computation of engineered features completed', 'Computation of raw features started', 'Computation of raw features complet

Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject,AutoML_032758d2-dde3-41ac-a05f-7a797dadf0d6_51,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
import pickle
best_remote_run,best_automl_model=remote_run.get_output()
#joblib.dump(value=best_automl_model,filename='automl_model.joblib')
pkl_filename = "automl_model.pkl"
with open(pkl_filename,'wb') as f:
    pickle.dump(best_automl_model,f)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
from azureml.core.model import Model
# Register model
model = Model.register(workspace = ws,
                        model_name = "Heart_failure_prediction_model",
                        model_path = "./automl_model.pkl",
                        description = "Best AutoML Model",)

Registering model Heart_failure_prediction_model


TODO: In the cell below, send a request to the web service you deployed to test it.

In [12]:
%%writefile amlscore.py
import json
import numpy as np
import os
import pickle
import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("automl")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        result = model.predict(data)
        return result.tolist()
    except Exception as ex:
        error = str(ex)
        return error

Overwriting amlscore.py


TODO: In the cell below, print the logs of the web service and delete the service

In [13]:
from azureml.automl.core.shared import constants
best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, 'azureml-automl.yml')


In [14]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script="scoring.py")

In [15]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)
print(deployment_config)

In [16]:
best_remote_run, model = remote_run.get_output()
best_remote_run_metrics = best_remote_run.get_metrics()

print(model)
print('Best Run Id: ', best_remote_run.id)
print('\n Accuracy:', best_remote_run_metrics['accuracy'])

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('25', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=1, colsample_bytree=1, eta=0.5, gamma=0.01, max_depth=7, max_leaves=15, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.5208333333333334, reg_lambda=2.291666666666667, subsample=1, tree_method='auto'))], verbose=False)), ('27',

In [17]:
import joblib
joblib.dump(model,'outputs/automl_model.pkl')
print(best_remote_run.get_tags())

{'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":0,"CurrentNodeCount":1}', 'mlflow.source.type': 'JOB', 'mlflow.source.name': 'automl_driver.py', '_aml_system_codegen': 'completed', '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True', 'model_explain_run_id': 'AutoML_032758d2-dde3-41ac-a05f-7a797dadf0d6_ModelExplain', 'model_explanation': 'True'}


In [18]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ="outputs/automl_model.pkl",
                        model_name = "automl")

Registering model automl


In [19]:
%%writefile amlscore.py
import json
import numpy as np
import os
import pickle
import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("automl")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        result = model.predict(data)
        return result.tolist()
    except Exception as ex:
        error = str(ex)
        return error

Overwriting amlscore.py


In [20]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.automl.core.shared import constants
from azureml.core.conda_dependencies import CondaDependencies



best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, "deploy.yml")
deployenv = Environment.from_conda_specification(name="deployenv", file_path="deploy.yml")

inference_config = InferenceConfig(entry_script='amlscore.py', environment=deployenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=1, description="heart-failure-model-final-one")

service = Model.deploy(ws, "automl-capstone", [model], inference_config, aciconfig)

service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-04-23 11:04:04+00:00 Creating Container Registry if not exists.
2023-04-23 11:04:04+00:00 Registering the environment.
2023-04-23 11:04:06+00:00 Use the existing image.
2023-04-23 11:04:06+00:00 Generating deployment configuration.
2023-04-23 11:04:06+00:00 Submitting deployment to compute.
2023-04-23 11:04:13+00:00 Checking the status of deployment automl-capstone..
2023-04-23 11:06:17+00:00 Checking the status of inference endpoint automl-capstone.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [21]:
print(service.get_logs())


/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-04-23T11:05:52,161103200+00:00 - iot-server/run 
2023-04-23T11:05:52,161126700+00:00 - rsyslog/run 
2023-04-23T11:05:52,167277900+00:00 - gunicorn/run 
bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by bash)
2023-04-23T11:05:52,173911300+00:00 | gunicorn/run | 
2023-04-23T11:05:52,179003700+00:00 | gunicorn/run | ####################################

In [22]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://b1940299-8471-4fb7-a4d9-eb8a56e60514.germanywestcentral.azurecontainer.io/score

swagger URI: 
http://b1940299-8471-4fb7-a4d9-eb8a56e60514.germanywestcentral.azurecontainer.io/swagger.json

http://b1940299-8471-4fb7-a4d9-eb8a56e60514.germanywestcentral.azurecontainer.io/score
http://b1940299-8471-4fb7-a4d9-eb8a56e60514.germanywestcentral.azurecontainer.io/swagger.json


In [23]:
import requests
import json

data={"data":
  [{'age': 81.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 781,
  'diabetes': 0,
  'ejection_fraction': 45,
  'high_blood_pressure': 1,
  'platelets': 218000,
  'serum_creatinine': 2.5,
  'serum_sodium': 127,
  'sex': 0,
  'smoking': 0,
  'time': 2}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())

The output is:  [1]


In [24]:
result.status_code


200

In [25]:
#logs of the webservice
logs=service.get_logs()
print(logs)

/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-04-23T11:05:52,161103200+00:00 - iot-server/run 
2023-04-23T11:05:52,161126700+00:00 - rsyslog/run 
2023-04-23T11:05:52,167277900+00:00 - gunicorn/run 
bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by bash)
2023-04-23T11:05:52,173911300+00:00 | gunicorn/run | 
2023-04-23T11:05:52,179003700+00:00 | gunicorn/run | ####################################